In [1]:
import numpy as np
import tqdm
xx, yy = np.meshgrid(np.linspace(-3, 3, 500), np.linspace(-3, 3, 500))
np.random.seed(0)
X = np.random.randn(500, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)


In [4]:
import torch
import torch.nn as nn

In [5]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
y = y*2 - 1

In [7]:
class KernelSVC():
    def __init__(self, n_samples, C=0.5, gamma=50, iters=50000, lr=1e-5):
        self.n_samples = n_samples
        self.C = C
        self.gamma = gamma
        self.alpha = nn.Parameter(torch.rand(n_samples))
        self.iters = iters
        self.lr = lr
    
    def _loss(self, K, y):
        loss1 = -self.alpha.sum()
        loss2 = 0.5 * (torch.outer((self.alpha * y), (self.alpha * y)) * K).sum()
        loss3 = self.gamma * (torch.dot(self.alpha, y)**2)
        loss = loss1 + loss2 + loss3
        return loss
    
    def _zero_grad(self):
        if self.alpha.grad is not None:
            self.alpha.grad.zero_()
    
    def _update_weights(self):
        self.alpha.data -= self.lr * self.alpha.grad.data
    
    def _clip_weights(self):
        self.alpha.data = torch.clamp(self.alpha.data, min=0, max=self.C)
    
    def _kernel_matrix(self, u, v):
        in_dim = u.shape[1]
        rbf_coeff = 1 / in_dim
        
        dist = torch.cdist(u, v) ** 2
        K = torch.exp(-rbf_coeff * dist)
        return K
        
    def fit(self, X, y):
        K = self._kernel_matrix(X, X)
        for i in tqdm.tqdm(range(self.iters)):
            self._zero_grad()
            self._clip_weights()
            loss = self._loss(K, y)
            loss.backward()
            self._update_weights()
        
        with torch.no_grad():
            self._clip_weights()
            self.idx_w = torch.where(self.alpha.data>0)[0]
            self.idx_b = torch.where((self.alpha.data>0)&(self.alpha.data<self.C))[0]
            
            self.sup_X = X[self.idx_w].clone()
            self.sup_y = y[self.idx_w].clone()
            self.sup_alpha = self.alpha.data[self.idx_w].clone()
        
            X_b = X[self.idx_b]
            K_b = self._kernel_matrix(self.sup_X, X_b)
            self.b = (y[self.idx_b] - torch.matmul((self.sup_alpha * self.sup_y), K_b)).mean()
    
    def predict(self, data):    
        with torch.no_grad():
            K = self._kernel_matrix(self.sup_X, data)
            score = torch.matmul((self.sup_alpha * self.sup_y), K) + self.b
            pred = (score > 0).to(data.dtype)
            pred = pred * 2 - 1
        
        return pred, score

In [8]:
N = len(X)
clf = KernelSVC(N, C=10, gamma=50, iters=50000, lr=1e-5)

In [9]:
clf.fit(X, y)

100%|███████████████████████████████████| 50000/50000 [00:30<00:00, 1626.40it/s]
